# pdfの英語文献から，出現頻度順の日訳英単語リストを作成するプログラム
## INPUT : PDF 英語文献     
## OUTPUT : CSV 翻訳 word list

### 品詞の略については以下サイトを参照
[Qiita](https://qiita.com/m__k/items/ffd3b7774f2fde1083fa "NLTKの使い方をいろいろ調べてみた")

### 初期設定
#### 最初の一回のみ，実行する．その後はコメントアウトしてよい．

In [ ]:
# # 使用するライブラリのインストール
# !pip install PyPDF2
# !pip install nltk
# !pip install deep_translator
# !pip install tqdm

# # nltkライブラリの事前ダウンロードファイル
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [1]:
import PyPDF2

# INPUT: pdf_file に 読み込むパスを記述！！！

In [2]:
# pdf 読み込み，textに変換．
pdf_file = "../../../RNA Viral Community in Human Feces.pdf"
with open(pdf_file, "rb") as f:
    reader = PyPDF2.PdfFileReader(f)
    num_page = reader.getNumPages()
    pages = []
    for i in range(num_page):
        pages.append(reader.getPage(i).extractText())


### 単語の整理: 単語群をもっと整えたい場合は，以下を編集するとよいかも
### ほしい品詞なども設定できる

In [3]:
import os
import nltk
import numpy as np
import pandas as pd
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

word_lists = []
for i in range(num_page):
    word_list = nltk.word_tokenize(pages[i])
    word_lists.extend(word_list)

# いらない単語を削除
removed_word_list = []
for word in word_lists:
    if word.isalpha(): # アルファベットのみ使用
        if len(word) > 2: # 2文字より大きい文字列のみ使用
            removed_word_list.append(word)
            
            
# basic word list を読み込み，listに含まれる単語を削除する．
basic_words_total_df = pd.read_csv(os.getcwd() + '/basicwordlist/english-word-list-total.csv', header=3, sep=';')
basic_words_nouns_df = pd.read_csv(os.getcwd() + '/basicwordlist/english-word-list-nouns.csv', header=3, sep=';').rename(columns={'noun': 'word'})
basic_words_verbs_df = pd.read_csv(os.getcwd() + '/basicwordlist/english-word-list-verbs.csv', header=3, sep=';').rename(columns={'verb': 'word'})
basic_words_adjectives_df = pd.read_csv(os.getcwd() + '/basicwordlist/english-word-list-adjectives.csv', header=3, sep=';').rename(columns={'adjective': 'word'})
most_common_words_df = pd.read_csv(os.getcwd() + '/basicwordlist/most-common-words-1k.csv', header=0).rename(columns={'Word': 'word'})
eng_vocab10k_df = pd.read_csv(os.getcwd() + '/basicwordlist/eng_vocab10k.csv', header=None).rename(columns={0: 'word'})
print(basic_words_total_df.shape)
print(basic_words_nouns_df.shape)
print(basic_words_verbs_df.shape)
print(most_common_words_df.shape)
print(eng_vocab10k_df.shape)

basic_words_df = pd.concat([basic_words_total_df, basic_words_nouns_df, basic_words_verbs_df, basic_words_adjectives_df, most_common_words_df, eng_vocab10k_df], join='inner')

basic_words_df = basic_words_df.dropna(subset=['word'])
basic_words_df = basic_words_df.drop_duplicates(subset=['word']).reset_index()
print(basic_words_df.shape)
basic_words = basic_words_df['word'].to_list()
print(basic_words_df)

not_basic_word_list = []
for word in removed_word_list:
    if word.lower() not in basic_words:
        not_basic_word_list.append(word)


(508, 4)
(200, 4)
(200, 4)
(1000, 3)
(10000, 1)
(10017, 2)
       index    word
0          0     the
1          1     and
2          2      to
3          3      of
4          4       a
...      ...     ...
10012   9995    zope
10013   9996  zshops
10014   9997      zu
10015   9998     zum
10016   9999     zus

[10017 rows x 2 columns]


In [4]:
word_dict = {} # 重複回数をカウント
for word in not_basic_word_list:
    if not word in word_dict:
        word_dict[word] = 1
    else:
        word_dict[word] += 1
                

sorted_dict = sorted(word_dict.items(), key=lambda x:x[1], reverse=True)

# 品詞の取得
pos = nltk.pos_tag([pair[0] for pair in sorted_dict])

print(len(pos))
# 欲しい品詞のポジションを取得
p = []
for i, pair in enumerate(pos):
    po = pair[1]
    if any(x in po for x in ["FW", "JJ", "LS", "MD", "NN", "RB", "RP", "VB"]):
        p.append(i)

sorted_dict = np.array(sorted_dict)[p]
pos = np.array(pos)[p]

print(len(p))
print(sorted_dict.shape)
print(pos.shape)
print((pos))
print(sorted_dict)

789
787
(787, 2)
(787, 2)
[['PMMV' 'NNP']
 ['fecal' 'JJ']
 ['feces' 'NNS']
 ...
 ['Lachapelle' 'NNP']
 ['Milling' 'NNP']
 ['blender' 'NN']]
[['PMMV' '108']
 ['fecal' '50']
 ['feces' '33']
 ...
 ['Lachapelle' '1']
 ['Milling' '1']
 ['blender' '1']]


### 英日訳 1秒間隔で英日訳を取得
- 単語数が少ない場合は，sleepを消してもよい．
- 単語数が多く，sleepが短いと，接続がキャンセルされる．

In [5]:
from http.client import RemoteDisconnected
from deep_translator import GoogleTranslator
from tqdm.notebook import tqdm
from time import sleep

translator = GoogleTranslator(sourse='en', target='ja')

engs = []
for i, word in tqdm(enumerate(sorted_dict), total=len(sorted_dict)):
    try:
        translated = translator.translate(word[0])
        engs.append(translated)
        print("{}\t{}\t{}\t{}\t{}".format(sorted_dict[i][1], sorted_dict[i][0], pos[i][0], pos[i][1], translated))
        sleep(1)
    except:
        print(f"Error!!!\ti: {i}\tword: {word}")
        sleep(3)
        try:
            translated = translator.translate(word[0])
            engs.append(translated)
            print("{}\t{}\t{}\t{}\t{}".format(sorted_dict[i][1], sorted_dict[i][0], pos[i][0], pos[i][1], translated))
            sleep(1)
        except:
            print(f"Error!!!!!! 2times\ti: {i}\tword: {word}")
    

  0%|          | 0/787 [00:00<?, ?it/s]

108	PMMV	PMMV	NNP	PMMV
50	fecal	fecal	JJ	糞便
33	feces	feces	NNS	糞便
29	mosaic	mosaic	VBP	モザイク
18	PBV	PBV	NNP	PBV
16	mottle	mottle	NN	まだら
14	Feces	Feces	NNPS	糞便
13	sequencing	sequencing	VBG	シーケンシング
12	PLoS	PLoS	NNP	PLOS
11	gastroenteritis	gastroenteritis	NN	胃腸炎
11	DOI	DOI	NNP	土肥
11	clones	clones	NNS	クローン
11	PCR	PCR	NNP	PCR
10	chlorotic	chlorotic	JJ	クロロティック
10	identiﬁed	identiﬁed	NN	識別された
10	Abundant	Abundant	NNP	豊富
10	contigs	contigs	VBZ	コンティグ
10	primer	primer	JJR	プライマー
9	enteric	enteric	JJ	腸内
9	Nicotiana	Nicotiana	NNP	ニコチアナ
8	abundant	abundant	JJ	豊富
8	GenBank	GenBank	NNP	ジェンバンク
8	Grapevine	Grapevine	NNP	グレープバイン
8	phylogenetic	phylogenetic	JJ	系統学的
8	assay	assay	NN	アッセイ
8	Microbiol	Microbiol	NNP	微生物
7	shotgun	shotgun	NN	散弾銃
7	pathogenic	pathogenic	JJ	病原性の
7	Capsicum	Capsicum	NNP	トウガラシ属
7	ﬁve	ﬁve	NNP	五つ
7	TaqMan	TaqMan	NNP	タックマン
7	CCG	CCG	NNP	CCG
7	AGG	AGG	NNP	AGG
7	Virol	Virol	NNP	ビロル
6	uncultured	uncultured	VBD	培養されていない
6	tangential	tangential	JJ	接線
6	microbial	microbial	JJ	微生物
6	ﬂora	ﬂor

### 必要な配列をマージしておく 

In [12]:
import pandas as pd

counts = [i[1] for i in sorted_dict]
words = [i[0] for i in sorted_dict]
poss = [i[1] for i in pos]
eng_lower = [i[0].lower() for i in sorted_dict]
jp_lower = [i.lower() for i in engs]

result_df = pd.DataFrame(list(zip(counts, words, poss, engs, eng_lower, jp_lower)), columns=["出現回数", "英語", "品詞", "日訳", "eng_lower", "jp_lower"]) 

print(result_df)

    出現回数          英語   品詞      日訳   eng_lower jp_lower
0    108        PMMV  NNP    PMMV        pmmv     pmmv
1     50       fecal   JJ      糞便       fecal       糞便
2     33       feces  NNS      糞便       feces       糞便
3     29      mosaic  VBP    モザイク      mosaic     モザイク
4     18         PBV  NNP     PBV         pbv      pbv
..   ...         ...  ...     ...         ...      ...
782    1      Sharpe  NNP    シャープ      sharpe     シャープ
783    1    Bergener  NNP    ベルゲン    bergener     ベルゲン
784    1  Lachapelle  NNP   ラシャペル  lachapelle    ラシャペル
785    1     Milling  NNP  フライス加工     milling   フライス加工
786    1     blender   NN   ブレンダー     blender    ブレンダー

[787 rows x 6 columns]


### 以下の条件のものを削除
- 翻訳前と翻訳後が同じであるもの

In [13]:
print(result_df.shape)

result_df = result_df[~(result_df["eng_lower"]==result_df["jp_lower"])]

print(result_df.shape)
    

(787, 6)
(731, 6)


## OUTPUT: save_fileに保存先パスを記述！！！

In [15]:
save_file = os.path.splitext(pdf_file)[0] + "_word_list" + ".csv"

result_df.to_csv(save_file, encoding='utf-8-sig', header=True, index=False, columns=["出現回数", "英語", "品詞", "日訳"])